<a href="https://colab.research.google.com/github/zhaodan-npu/zhaodan-npu.github.io/blob/master/Simulation_Code_for_Figure_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
This script simulates the behavior of a coupled oscillator system (Kuramoto model)
under varying noise intensities and generates the data and plots for Figure 4 of the paper.

The model includes pairwise (K1) and higher-order (K2) coupling terms and is
subjected to Lévy stable noise. The script calculates the Probability Density
Function (PDF) of the order parameter and plots it as a heatmap.

Key Features:
- Performs numerical integration using the 4th-order Runge-Kutta (RK4) method.
- Accelerates computations using Numba.
- Allows customization of simulation parameters via command-line arguments.
- Saves the resulting PDF matrix to a .txt file.
- Saves the final PDF heatmap as a .png image.
"""

import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
from scipy.stats import gaussian_kde
from scipy.stats import levy_stable
import argparse
import time

# --- Core computation functions (accelerated with Numba) ---

@njit(parallel=True)
def kuramoto_derivatives(theta, omega, K1, K2):
    """
    Calculates the phase derivative d(theta)/dt for each oscillator in the Kuramoto model.

    Args:
    - theta (np.ndarray): A (R, N) array representing the current phases of N oscillators for R realizations.
    - omega (np.ndarray): A (R, N) array for the natural frequencies of each oscillator.
    - K1 (float): Pairwise coupling strength.
    - K2 (float): Higher-order coupling strength.

    Returns:
    - dtheta (np.ndarray): A (R, N) array of the phase derivatives for each oscillator.
    """
    R, N = theta.shape
    dtheta = np.empty_like(theta)

    for r in prange(R):
        # Calculate the mean-field components for each realization
        C = 0.0
        S = 0.0
        for j in range(N):
            C += np.cos(theta[r, j])
            S += np.sin(theta[r, j])

        for i in range(N):
            # Calculate the pairwise coupling term
            s_pair = 0.0
            for j in range(N):
                s_pair += np.sin(theta[r, j] - theta[r, i])
            pairwise = (K1 / N) * s_pair

            # Calculate the higher-order coupling term
            A = 0.0
            B = 0.0
            for j in range(N):
                X = 2 * theta[r, j] - theta[r, i]
                A += np.sin(X)
                B += np.cos(X)
            high_order = (K2 / (N * N)) * (C * A - S * B)

            dtheta[r, i] = omega[r, i] + pairwise + high_order

    return dtheta

@njit
def mod_2pi(theta):
    """Constrains the phases to the interval [0, 2*pi]."""
    R, N = theta.shape
    for r in range(R):
        for i in range(N):
            theta[r, i] = theta[r, i] % (2 * np.pi)

@njit
def rk4_step(theta, omega, K1, K2, dt):
    """
    Performs a single 4th-order Runge-Kutta (RK4) integration step.
    """
    R, N = theta.shape

    # k1
    k1 = kuramoto_derivatives(theta, omega, K1, K2)
    theta_temp = theta + 0.5 * dt * k1

    # k2
    k2 = kuramoto_derivatives(theta_temp, omega, K1, K2)
    theta_temp = theta + 0.5 * dt * k2

    # k3
    k3 = kuramoto_derivatives(theta_temp, omega, K1, K2)
    theta_temp = theta + dt * k3

    # k4
    k4 = kuramoto_derivatives(theta_temp, omega, K1, K2)

    # Final update
    theta_new = theta + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)
    mod_2pi(theta_new)
    return theta_new

@njit(parallel=True)
def order_parameter(theta):
    """
    Calculates the order parameter 'r' of the system.
    """
    R, N = theta.shape
    r_vals = np.empty(R)
    for r in prange(R):
        re = 0.0
        im = 0.0
        for i in range(N):
            re += np.cos(theta[r, i])
            im += np.sin(theta[r, i])
        r_vals[r] = np.sqrt((re / N) ** 2 + (im / N) ** 2)
    return r_vals

# --- Main simulation workflow ---

def simulate(theta0, omega, K1, K2, dt, T_steps, sigma, alpha, beta, transient_steps):
    """
    Executes the full simulation process.

    Args:
    - theta0 (np.ndarray): Initial phases.
    - omega (np.ndarray): Natural frequencies.
    - ... other simulation parameters.
    - transient_steps (int): Number of initial transient steps to discard.

    Returns:
    - r_all (np.ndarray): Recorded order parameter at each time step after the transient phase.
    """
    num_realizations, N = theta0.shape
    record_steps = T_steps - transient_steps

    if record_steps <= 0:
        raise ValueError("Total steps (T_steps) must be greater than transient_steps.")

    r_all = np.empty((num_realizations, record_steps))
    theta = theta0.copy()

    # Pre-calculate the noise scaling factor
    noise_scale = sigma * np.sqrt(dt)

    print("Starting simulation...")
    # Transient phase
    print(f"First, running {transient_steps} transient steps...")
    for t in range(transient_steps):
        theta = rk4_step(theta, omega, K1, K2, dt)
        noise = levy_stable.rvs(alpha, beta, size=(num_realizations, N)) * noise_scale
        theta += noise
        # mod_2pi is already called inside rk4_step, so it can be omitted here
        # unless the noise is very large.

    # Recording phase
    print(f"Transient phase complete. Starting to record for {record_steps} steps...")
    for t in range(record_steps):
        theta = rk4_step(theta, omega, K1, K2, dt)
        noise = levy_stable.rvs(alpha, beta, size=(num_realizations, N)) * noise_scale
        theta += noise
        r_all[:, t] = order_parameter(theta)

    print("Simulation complete.")
    return r_all

def main(args):
    """
    Main function: sets up parameters, runs the simulation, and generates results.
    """
    start_time = time.time()

    # --- 1. Initialize parameters ---
    print("Initializing parameters...")
    theta_initial = 2 * np.pi * np.random.rand(args.num_realizations, args.N)
    omega_values = np.random.standard_cauchy((args.num_realizations, args.N))

    sigma_vals = np.arange(args.sigma_start, args.sigma_end, args.sigma_step)
    x_vals = np.linspace(0, 1, 200)
    pdf_matrix = np.zeros((len(sigma_vals), len(x_vals)))

    # --- 2. Loop over different noise intensities and run simulations ---
    for i, sigma in enumerate(sigma_vals):
        print("-" * 50)
        print(f"Processing group {i+1}/{len(sigma_vals)}: sigma = {sigma:.2f}")

        r_all = simulate(
            theta_initial, omega_values, args.K1, args.K2, args.dt,
            args.T_steps, sigma, args.alpha, args.beta, args.transient_steps
        )

        # --- 3. Calculate and store the PDF ---
        print("Calculating PDF...")
        all_r = r_all.flatten()
        # Calculate PDF using Gaussian Kernel Density Estimation
        density = gaussian_kde(all_r)
        pdf_matrix[i, :] = density(x_vals)

    # --- 4. Save data and plot ---
    print("-" * 50)
    print(f"All simulations finished. Saving results...")

    # Save the PDF matrix data
    np.savetxt(args.output_file, pdf_matrix)
    print(f"PDF data saved to: {args.output_file}")

    # Plot and save the heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(pdf_matrix, extent=[x_vals.min(), x_vals.max(), sigma_vals.min(), sigma_vals.max()],
               aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(label='PDF')
    plt.xlabel("Order Parameter r")
    plt.ylabel("Noise Intensity $\sigma$")
    plt.title(f"PDF of Order Parameter r (alpha={args.alpha}, K1={args.K1}, K2={args.K2})")
    plt.savefig(args.plot_file, dpi=300, bbox_inches='tight')
    print(f"Heatmap plot saved to: {args.plot_file}")
    # Uncomment the line below to display the plot locally during runtime
    # plt.show()

    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds")

if __name__ == '__main__':
    # --- Command-line argument parsing ---
    # This allows users to change parameters from the command line without editing the code.
    parser = argparse.ArgumentParser(description="Simulates a coupled oscillator system and generates a PDF heatmap of the order parameter.")

    # Physical parameters
    parser.add_argument('-N', type=int, default=100, help='Number of oscillators')
    parser.add_-argument('--K1', type=float, default=0.8, help='Pairwise coupling strength')
    parser.add_argument('--K2', type=float, default=8.0, help='Higher-order coupling strength')

    # Simulation parameters
    parser.add_argument('--num_realizations', type=int, default=10, help='Number of realizations')
    parser.add_argument('--dt', type=float, default=0.01, help='Time step for integration')
    parser.add_argument('--T_steps', type=int, default=int(1e6), help='Total simulation steps')
    parser.add_argument('--transient_steps', type=int, default=int(1e5), help='Transient steps to discard')

    # Noise parameters
    parser.add_argument('--alpha', type=float, default=1.6, help='Alpha parameter for the Lévy stable distribution')
    parser.add_argument('--beta', type=float, default=0.0, help='Beta parameter for the Lévy stable distribution')

    # Sigma scan parameters
    parser.add_argument('--sigma_start', type=float, default=0.1, help='Start value for the sigma scan')
    parser.add_argument('--sigma_end', type=float, default=2.1, help='End value for the sigma scan')
    parser.add_argument('--sigma_step', type=float, default=0.5, help='Step size for the sigma scan')

    # File output parameters
    parser.add_argument('--output_file', type=str, default="pdf_matrix_sigma.txt", help='Filename for the saved PDF matrix data')
    parser.add_argument('--plot_file', type=str, default="pdf_heatmap_sigma.png", help='Filename for the saved heatmap plot')

    # Parse arguments and pass them to the main function
    parsed_args = parser.parse_args()
    main(parsed_args)